In [119]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader
from torchvision import transforms

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")


In [120]:
# transform_train = transforms.Compose([transforms.ToTensor()])
transform_train = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.137,), (0.3081,))])
transform_valid = transforms.Compose([transforms.ToTensor()])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                      download=True, transform=transform_train)#, target_transform=lambda x: torch.Tensor([x]).float())
trainloader = DataLoader(trainset, batch_size=20,
                         shuffle=True, num_workers=0)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                     download=True, transform=transform_valid) #target_transform=lambda x: torch.Tensor([x]).float())
testloader = DataLoader(testset, batch_size=20,
                        shuffle=False, num_workers=0)


In [140]:
class Cnn(nn.Module):
    def __init__(self):
        super(Cnn, self).__init__()
        self.conv_blocks_1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.conv_blocks_2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.dropout = nn.Dropout(0.5)
        self.flatten = nn.Flatten(1)
        self.full_connection1 = nn.Sequential(
            nn.Linear(7 * 7 * 64, 1024),
            nn.ReLU()
        )
        self.out = nn.Linear(1024, 10)

    def forward(self, x):
        x = self.conv_blocks_1(x)
        x = self.conv_blocks_2(x)
        x = self.flatten(x)
        x = self.full_connection1(x)
        x = self.dropout(x)
        x = self.out(x)
        return x


def metric_func(y_pred, y_true): return roc_auc_score(y_true=y_true.data.numpy(), y_score=y_pred.data.numpy())


matric_name = "auc"


def train_step(model: nn.Module, features: torch.Tensor, labels: torch.Tensor):
    features = features.to(device=device)
    labels = labels.to(device)
    optimizer.zero_grad()
    y_pred = model(features)
    loss = criterion(y_pred, labels)
    # metric = metric_func(y_pred, labels)
    metric = torch.Tensor(0)
    loss.backward()
    optimizer.step()
    return loss, 0


In [141]:
from sklearn.metrics import roc_auc_score

cnn = Cnn()
cnn.to(device)
criterion = nn.CrossEntropyLoss()
# criterion = nn.NLLLoss()
optimizer = torch.optim.SGD(cnn.parameters(), lr=0.1)

In [142]:
import torchkeras
input_shape = (1, 28, 28)
torchkeras.summary(Cnn(), input_shape=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
              ReLU-2           [-1, 32, 28, 28]               0
         MaxPool2d-3           [-1, 32, 14, 14]               0
            Conv2d-4           [-1, 64, 14, 14]          18,496
              ReLU-5           [-1, 64, 14, 14]               0
         MaxPool2d-6             [-1, 64, 7, 7]               0
           Flatten-7                 [-1, 3136]               0
            Linear-8                 [-1, 1024]       3,212,288
              ReLU-9                 [-1, 1024]               0
          Dropout-10                 [-1, 1024]               0
           Linear-11                   [-1, 10]          10,250
Total params: 3,241,354
Trainable params: 3,241,354
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.002991


In [143]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('./graph/tensorboard')
writer.add_graph(Cnn(),input_to_model = torch.rand(100, 1,28,28))
writer.close()

In [124]:
(features, labels) = next(iter(trainloader))
# print((features[0], labels[0]))
print(trainloader.batch_sampler)
print(len(trainloader))

3000


In [125]:
for epoch in range(20):
    loss_sum = []
    metric_sum = 0.0
    for i, data in enumerate(trainloader):
        (features, labels) = data
        loss, metric = train_step(cnn, features=features, labels=labels)
        loss_sum.append(loss)
        metric_sum += metric
        loss_sum.append(loss.item())
        if i % 100 == 0:
            print('[epoch: %d, id: %5d, %3d%%] loss: %.3f' %
                  (epoch + 1, i + 1, 100 * i / len(trainloader), torch.mean(torch.Tensor(loss_sum))))


[epoch: 1, id:     1,   0%] loss: 2.310
[epoch: 1, id:   101,   3%] loss: 1.028
[epoch: 1, id:   201,   6%] loss: 0.681
[epoch: 1, id:   301,  10%] loss: 0.527
[epoch: 1, id:   401,  13%] loss: 0.441
[epoch: 1, id:   501,  16%] loss: 0.382
[epoch: 1, id:   601,  20%] loss: 0.346
[epoch: 1, id:   701,  23%] loss: 0.317
[epoch: 1, id:   801,  26%] loss: 0.294
[epoch: 1, id:   901,  30%] loss: 0.275
[epoch: 1, id:  1001,  33%] loss: 0.258
[epoch: 1, id:  1101,  36%] loss: 0.243
[epoch: 1, id:  1201,  40%] loss: 0.232
[epoch: 1, id:  1301,  43%] loss: 0.220
[epoch: 1, id:  1401,  46%] loss: 0.210
[epoch: 1, id:  1501,  50%] loss: 0.204
[epoch: 1, id:  1601,  53%] loss: 0.196
[epoch: 1, id:  1701,  56%] loss: 0.190
[epoch: 1, id:  1801,  60%] loss: 0.183
[epoch: 1, id:  1901,  63%] loss: 0.178
[epoch: 1, id:  2001,  66%] loss: 0.173
[epoch: 1, id:  2101,  70%] loss: 0.169
[epoch: 1, id:  2201,  73%] loss: 0.165
[epoch: 1, id:  2301,  76%] loss: 0.162
[epoch: 1, id:  2401,  80%] loss: 0.158
